# Test daily reward loss
Test the new model to check if it is working correctly.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os 
#os.environ['MPLCONFIGDIR'] = "$HOME"
#os.envir
# on["WANDB_DIR"] = os.path.join(os.path.dirname(__file__), "..", "wandb")
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), os.path.pardir)))
# sys append 
sys.path.append("..")
from data import BavarianCrops, BreizhCrops, SustainbenchCrops, ModisCDL
from torch.utils.data import DataLoader
from models.earlyrnn import EarlyRNN
from models.daily_earlyrnn import DailyEarlyRNN
import torch
from tqdm import tqdm
from utils.losses.early_reward_loss import EarlyRewardLoss
from utils.losses.stopping_time_proximity_loss import StoppingTimeProximityLoss, sample_three_uniform_numbers
from utils.losses.daily_reward_loss import DailyRewardLoss, log_class_prob_at_t_plus_zt
import sklearn.metrics
import pandas as pd
import wandb
from utils.plots import plot_label_distribution_datasets, boxplot_stopping_times
from utils.doy import get_doys_dict_test, get_doy_stop, create_sorted_doys_dict_test, get_approximated_doys_dict
from utils.helpers_training import parse_args_sweep, train_epoch
from utils.helpers_testing import test_epoch
from utils.metrics import harmonic_mean_score
from models.model_helpers import count_parameters
import matplotlib.pyplot as plt


## Test the helper functions for the loss

In [3]:
# craete a torch tensor of size (batchsize, sequencelength, nclasses) with random values
batchsize = 1
sequencelength = 5
nclasses = 4

log_class_probabilities = torch.rand((batchsize, sequencelength, nclasses))

# create a tensor of size (batchsize, sequencelength) with random values between 0 and 364
timestamps_left = torch.randint(0, sequencelength-1, (batchsize, sequencelength))


print("log_class_probabilities\n", log_class_probabilities)
print("timestamps_left\n", timestamps_left)

log_class_probab_at_t_plus_zt = log_class_prob_at_t_plus_zt(log_class_probabilities, timestamps_left)
print("log_class_probab_at_t_plus_zt\n", log_class_probab_at_t_plus_zt)

log_class_probabilities
 tensor([[[0.0256, 0.5144, 0.7861, 0.1210],
         [0.0609, 0.9291, 0.9235, 0.2702],
         [0.3193, 0.5586, 0.1668, 0.7046],
         [0.8062, 0.2824, 0.4670, 0.8571],
         [0.2979, 0.1941, 0.9765, 0.2255]]])
timestamps_left
 tensor([[1, 1, 0, 1, 1]])
log_class_probab_at_t_plus_zt
 tensor([[[0.0609, 0.9291, 0.9235, 0.2702],
         [0.3193, 0.5586, 0.1668, 0.7046],
         [0.3193, 0.5586, 0.1668, 0.7046],
         [0.2979, 0.1941, 0.9765, 0.2255],
         [0.2979, 0.1941, 0.9765, 0.2255]]])


## Test the loss

In [4]:
# config 
class Config():
    def __init__(self):
        self.alpha = 0.6
        self.backbonemodel = "LSTM"
        self.batchsize = 256
        self.corrected = True
        self.dataroot = os.path.join(os.environ.get("HOME", os.environ.get("USERPROFILE")),"elects_data")
        self.dataset = "breizhcrops"
        self.device = "cuda"
        self.epochs = 100
        self.epsilon = 10
        self.extra_padding_list = [0]
        self.hidden_dims = 64
        self.learning_rate = 0.001
        self.loss_weight = "balanced"
        self.patience = 30
        self.resume = False
        self.sequencelength = 365
        self.validation_set = "valid"
        self.weight_decay = 0
        self.daily_timestamps = True
        self.original_time_serie_lengths = [102]
        self.loss = "daily_reward_loss"
        
config = Config()

In [5]:
dataroot = os.path.join(config.dataroot,"breizhcrops")
input_dim = 13
test_ds = BreizhCrops(root=dataroot,partition=config.validation_set, sequencelength=config.sequencelength, corrected=config.corrected, daily_timestamps=config.daily_timestamps, original_time_serie_lengths=config.original_time_serie_lengths)
train_ds = BreizhCrops(root=dataroot,partition="train", sequencelength=config.sequencelength, corrected=config.corrected, daily_timestamps=config.daily_timestamps, original_time_serie_lengths=config.original_time_serie_lengths)
nclasses = train_ds.nclasses
class_names = train_ds.labels_names
traindataloader = DataLoader(train_ds,batch_size=config.batchsize)
testdataloader = DataLoader(test_ds, batch_size=config.batchsize)

2559635960 2559635960


loading data into RAM: 100%|██████████| 67523/67523 [00:31<00:00, 2157.37it/s]


2253658856 2253658856


loading data into RAM: 100%|██████████| 85310/85310 [00:33<00:00, 2554.71it/s]


cost function: 

model: 

In [6]:
model = DailyEarlyRNN(config.backbonemodel, nclasses=nclasses, input_dim=input_dim, sequencelength=config.sequencelength, hidden_dims=config.hidden_dims).to(config.device)


optimizer

In [7]:
# exclude decision head linear bias from weight decay
decay, no_decay = list(), list()
for name, param in model.named_parameters():
    if name == "stopping_decision_head.projection.0.bias":
        no_decay.append(param)
    else:
        decay.append(param)

optimizer = torch.optim.AdamW([{'params': no_decay, 'weight_decay': 0, "lr": config.learning_rate}, {'params': decay}],
                                lr=config.learning_rate, weight_decay=config.weight_decay)



loss: 

In [8]:
if config.loss_weight == "balanced":
    class_weights = train_ds.get_class_weights().to(config.device)
else: 
    class_weights = None

if config.loss == "early_reward":
    criterion = EarlyRewardLoss(alpha=config.alpha, epsilon=config.epsilon, weight=class_weights)
elif config.loss == "stopping_time_proximity":
    criterion = StoppingTimeProximityLoss(alphas=config.alphas, weight=class_weights)
elif config.loss == "daily_reward_loss":
    criterion = DailyRewardLoss(alpha=config.alpha, weight=class_weights)

Train example

In [12]:
# ----------------------------- TRAINING -----------------------------
start_epoch = 1
print("starting training...")
with tqdm(range(start_epoch, config.epochs + 1)) as pbar:
    for epoch in pbar:
        trainloss = train_epoch(model, traindataloader, optimizer, criterion, device=config.device)
        testloss, stats = test_epoch(model, testdataloader, criterion, config.device, return_id=test_ds.return_id, daily_timestamps=config.daily_timestamps)

        # statistic logging and visualization...
        precision, recall, fscore, support = sklearn.metrics.precision_recall_fscore_support(
            y_pred=stats["predictions_at_t_stop"][:, 0], y_true=stats["targets"][:, 0], average="macro",
            zero_division=0)
        accuracy = sklearn.metrics.accuracy_score(
            y_pred=stats["predictions_at_t_stop"][:, 0], y_true=stats["targets"][:, 0])
        kappa = sklearn.metrics.cohen_kappa_score(
            stats["predictions_at_t_stop"][:, 0], stats["targets"][:, 0])

        classification_loss = stats["classification_loss"].mean()
        earliness_reward = stats["earliness_reward"].mean()
        earliness = 1 - (stats["t_stop"].mean() / (config.sequencelength - 1))
        harmonic_mean = harmonic_mean_score(accuracy, stats["classification_earliness"])

starting training...


  1%|          | 1/100 [00:53<1:28:23, 53.57s/it]


KeyboardInterrupt: 